<a href="https://colab.research.google.com/github/EduardoMA7/Sistema-Inteligente-basado-en-CNN-para-el-diagnostico-de-cancer-de-pulmon/blob/main/cnn_pulmon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Requerimientos**

In [ ]:
!pip install streamlit tensorflow keras matplotlib seaborn pandas numpy scikit-learn pyngrok fpdf2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['NGROK_TOKEN'] = "TOKEN"

# **Modulo de modelos (model_utils.py)**

In [ ]:
%%writefile model_utils.py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D,
    Dense, Dropout, BatchNormalization
)
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.applications import DenseNet121, ResNet50

def load_models():
    models = {}

    # ---- Modelo 1: CNN 2D Personalizada (1 canal) ----
    input_shape = (128, 128, 1)
    inputs = Input(input_shape)
    x = Conv2D(32, (3,3), activation='relu')(inputs)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3,3), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    models['CNN 2D Personalizada'] = model

    # ---- Modelo 2: DenseNet121 (3 canales) ----
    densenet_input = Input(shape=(128, 128, 3))
    base_densenet = DenseNet121(weights='imagenet', include_top=False, input_tensor=densenet_input)
    base_densenet.trainable = False

    x = base_densenet.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    densenet_model = Model(inputs=densenet_input, outputs=output)
    densenet_model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    models["DenseNet121"] = densenet_model

    # ---- Modelo 3: ResNet50 (3 canales) ----
    resnet_input = Input(shape=(128, 128, 3))
    base_resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=resnet_input)
    base_resnet.trainable = False

    x = base_resnet.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    resnet_model = Model(inputs=resnet_input, outputs=output)
    resnet_model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')]
    )
    models["ResNet50"] = resnet_model

    return models

def predict_image(model, image):
    if len(image.shape) == 2:
        image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)
    confidence = float(np.max(prediction))
    diagnosis = 1 if prediction > 0.5 else 0
    heatmap = generate_saliency_map(model, image[0])

    return diagnosis, confidence, heatmap

def predict_image_rgb(model, image_gray):
    if len(image_gray.shape) == 2:
        image_gray = np.expand_dims(image_gray, axis=-1)

    image_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(image_gray)).numpy()
    image_rgb_batch = np.expand_dims(image_rgb, axis=0)

    prediction = model.predict(image_rgb_batch)
    confidence = float(prediction[0][0])
    diagnosis = 1 if confidence > 0.5 else 0
    heatmap = generate_saliency_map(model, image_rgb)

    return diagnosis, confidence, heatmap

def generate_saliency_map(model, image):
    if len(image.shape) == 3:
        image_tensor = tf.convert_to_tensor(np.expand_dims(image, axis=0), dtype=tf.float32)
    elif len(image.shape) == 4:
        image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
    else:
        raise ValueError("Imagen debe tener 3 (HWC) o 4 (BHWC) dimensiones.")

    with tf.GradientTape() as tape:
        tape.watch(image_tensor)
        prediction = model(image_tensor)

    gradients = tape.gradient(prediction, image_tensor)
    saliency_map = tf.reduce_max(tf.abs(gradients), axis=-1)[0].numpy()

    if saliency_map.max() != saliency_map.min():
        saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())
    else:
        saliency_map = np.zeros_like(saliency_map)

    return saliency_map

# **Módulo de Preprocesamiento (preprocessing.py)**

In [ ]:
%%writefile preprocessing.py
from skimage.io import imread
from skimage.transform import resize
from skimage.exposure import equalize_adapthist

def preprocess_image(image, target_size=(128, 128)):
    image = resize(image, target_size, mode='reflect', anti_aliasing=True)
    image = equalize_adapthist(image)
    image = (image - image.min()) / (image.max() - image.min() + 1e-8)

    return image

def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = imread(image_path, as_gray=True)
    image_original = resize(image, target_size, mode='reflect', anti_aliasing=True)
    image_preprocessed = preprocess_image(image, target_size)

    return image_preprocessed, image_original

In [ ]:
#Importa dataset de drive -> Preprocesa -> Guarda en formato .npy
import os
import numpy as np
from preprocessing import preprocess_image
from skimage.io import imread
from tqdm import tqdm

def preprocess_and_save_dataset_npy(input_dir, output_dir, target_size=(128, 128)):
    os.makedirs(output_dir, exist_ok=True)
    classes = ['cancer', 'no_cancer']

    for label in classes:
        input_class_dir = os.path.join(input_dir, label)
        output_class_dir = os.path.join(output_dir, label)
        os.makedirs(output_class_dir, exist_ok=True)

        print(f"Procesando clase: {label} en {input_dir}")
        for file_name in tqdm(os.listdir(input_class_dir)):
            input_path = os.path.join(input_class_dir, file_name)
            file_base = os.path.splitext(file_name)[0]
            output_path = os.path.join(output_class_dir, file_base + ".npy")

            try:
                image = imread(input_path, as_gray=True)
                processed_image = preprocess_image(image, target_size)
                np.save(output_path, processed_image.astype(np.float32))
            except Exception as e:
                print(f"Error con {file_name}: {e}")

if __name__ == "__main__":
    base_input_dir = "/content/drive/MyDrive/lung_cancer_MRI_dataset"
    base_output_dir = "/content/processed_lung_dataset_npy"

    for subset in ['train', 'validate']:
        input_dir = os.path.join(base_input_dir, subset)
        output_dir = os.path.join(base_output_dir, subset)
        preprocess_and_save_dataset_npy(input_dir, output_dir)

# **Módulo de Entrenamiento (train.py)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
    TensorBoard
)
from model_utils import load_models
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from skimage.io import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

def load_dataset(data_dir, test_size=0.2, random_state=42):
    cancer_cases = [os.path.join(data_dir, 'cancer', f) for f in os.listdir(os.path.join(data_dir, 'cancer'))]
    no_cancer_cases = [os.path.join(data_dir, 'no_cancer', f) for f in os.listdir(os.path.join(data_dir, 'no_cancer'))]

    cancer_labels = [1] * len(cancer_cases)
    no_cancer_labels = [0] * len(no_cancer_cases)

    all_cases = cancer_cases + no_cancer_cases
    all_labels = cancer_labels + no_cancer_labels

    train_cases, val_cases, train_labels, val_labels = train_test_split(
        all_cases, all_labels, test_size=test_size, random_state=random_state, stratify=all_labels
    )

    return train_cases, val_cases, train_labels, val_labels

class LungImageGenerator(tf.keras.utils.Sequence):
    def __init__(self, cases, labels, batch_size=32, image_size=(128, 128), augment=False, rgb=None, model_name=None, **kwargs):
        super().__init__(**kwargs)
        self.cases = cases
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        self.model_name = model_name
        self.indices = np.arange(len(self.cases))

        if rgb is not None:
            self.rgb = rgb
        elif self.model_name in ["DenseNet121", "ResNet50"]:
            self.rgb = True
        else:
            self.rgb = False

        if self.augment:
            self.augmenter = ImageDataGenerator(
                rotation_range=10,
                width_shift_range=0.05,
                height_shift_range=0.05,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode='reflect'
            )
        else:
            self.augmenter = None

    def __len__(self):
        return int(np.ceil(len(self.cases) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_cases = [self.cases[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]

        batch_images = []
        for path in batch_cases:
          #image = imread(path, as_gray=True)
          image = np.load(path).astype(np.float32)
          if len(image.shape) == 2:
            image = np.expand_dims(image, axis=-1)

          if self.rgb:
            image = tf.image.grayscale_to_rgb(tf.convert_to_tensor(image)).numpy()
            #image = image.astype('float32') / 255.0

          batch_images.append(image)

        batch_images = np.array(batch_images, dtype=np.float32)
        batch_labels = np.array(batch_labels, dtype=np.float32)

        if self.augment:
            aug_iter = self.augmenter.flow(batch_images, batch_labels, batch_size=self.batch_size, shuffle=False)
            batch_images, batch_labels = next(aug_iter)

        return batch_images, batch_labels

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

def train_model(model, train_generator, val_generator, epochs, model_name='lung_cancer_model', class_weight=None):
    callbacks = [
        ModelCheckpoint(
            f'models/{model_name}.keras',
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        TensorBoard(
            log_dir=f'logs/{model_name}_{datetime.now().strftime("%Y%m%d-%H%M%S")}',
            histogram_freq=1
        )
    ]

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weight,
        verbose=1
    )

    pd.DataFrame(history.history).to_csv(f'models/{model_name}_history.csv', index=False)

    return history

if __name__ == "__main__":
    data_dir = '/content/processed_lung_dataset_npy/train'
    os.makedirs('models', exist_ok=True)
    os.makedirs('logs', exist_ok=True)

    train_cases, val_cases, train_labels, val_labels = load_dataset(data_dir)

    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_labels),
        y=train_labels
    )
    class_weight = dict(enumerate(class_weights))

    models = load_models()
    modelo_objetivo = "DenseNet121"
    models = {k: v for k, v in models.items() if k == modelo_objetivo}

    train_gen = LungImageGenerator(train_cases, train_labels, batch_size=32, augment=True, model_name=modelo_objetivo)
    val_gen = LungImageGenerator(val_cases, val_labels, batch_size=32, augment=False, model_name=modelo_objetivo)

    for name, model in models.items():
        print(f"\nEntrenando modelo: {name}")
        history = train_model(
            model,
            train_gen,
            val_gen,
            epochs=50,
            model_name=f"{name.lower().replace(' ', '_')}_lung_cancer",
            class_weight=class_weight
        )

# **Traducciones (translations.py)**


In [ ]:
%%writefile translations.py
app_text = {
    "Español": {
        "page_title": "Sistema de Diagnóstico de Cáncer de Pulmón",
        "page_info": "🏥",
        "nav_diagnosis": "Diagnóstico por Imagen",
        "nav_eval": "Evaluación de Modelos",
        "upload_header": "Carga de Imagen",
        "upload_option": "Seleccione el tipo de entrada",
        "upload_png": "Subir imagen PNG",
        "upload_example": "Usar ejemplo",
        "select_class": "Seleccione clase de ejemplo",
        "select_image": "Seleccione imagen",
        "image_warning": "No se encontraron imágenes en la carpeta",
        "model_warning": "No se encontró el modelo",
        "image_view": "Visualización de Imagen",
        "original_image": "Imagen Original Redimensionada",
        "preprocessed_image": "Imagen Preprocesada",
        "results": "Resultados del Diagnóstico",
        "prediction": "Predicción",
        "confidence": "Confianza",
        "low_confidence": "La confianza es baja. Se recomienda evaluación adicional.",
        "negative": "No se detectaron signos malignos. Seguimiento rutinario recomendado.",
        "positive": "Posibles signos de cáncer detectados. Consulte a un especialista.",
        "saliency_map": "Mapa de Saliencia (Regiones relevantes)",
        "generate_pdf": "Generar Reporte PDF",
        "download_pdf": "📄 Descargar Reporte PDF",
        "eval_title": "Evaluación Comparativa de Modelos",
        "eval_description": "Esta sección permite evaluar los modelos con el dataset de validación y comparar sus métricas.",
        "generate_report_saved": "📄 Generar Reporte Comparativo PDF con Resultados Guardados",
        "download_comparative_pdf": "📄 Descargar PDF Comparativo",
        "re_eval": "¿Deseas volver a evaluar los modelos?",
        "re_eval_button": "🔍 Evaluar Modelos y Generar Reporte Comparativo",
        "eval_success": "¡Evaluación completada y reporte generado con éxito!",
        "metrics_header": "📊 Métricas por Modelo",
        "comparison_header": "📈 Comparaciones Estadísticas (McNemar)",
        "conf_matrix": "🧮 Matrices de Confusión",
        "roc_pr": "📉 Curvas ROC y Precision-Recall",
        "select_model": "Modelo a utilizar",
        "confidence_threshold": "Umbral de confianza para diagnóstico",
        "processing_image": "Procesando imagen...",
        "analyzing_image": "Analizando imagen...",
        "generate_pdf_button": "📄 Generar y Descargar Reporte PDF",
        "generate_from_saved_button": "📥 Generar PDF desde Resultados Guardados",
        "evaluating_models": "Evaluando modelos...",
        "sidebar_title": "Navegación",
        "sidebar_go_to": "Ir a",
        "model": "Modelo"
    },
    "English": {
        "page_title": "Lung Cancer Diagnosis System",
        "page_info": "🏥",
        "nav_diagnosis": "Image Diagnosis",
        "nav_eval": "Model Evaluation",
        "upload_header": "Image Upload",
        "upload_option": "Select input type",
        "upload_png": "Upload PNG image",
        "upload_example": "Use example",
        "select_class": "Select example class",
        "select_image": "Select image",
        "image_warning": "No images found in folder",
        "model_warning": "Model not found",
        "image_view": "Image Visualization",
        "original_image": "Original Resized Image",
        "preprocessed_image": "Preprocessed Image",
        "results": "Diagnosis Results",
        "prediction": "Prediction",
        "confidence": "Confidence",
        "low_confidence": "Low confidence. Additional evaluation recommended.",
        "negative": "No malignant signs detected. Routine follow-up recommended.",
        "positive": "Possible signs of cancer detected. Consult a specialist.",
        "saliency_map": "Saliency Map (Relevant regions)",
        "generate_pdf": "Generate PDF Report",
        "download_pdf": "📄 Download PDF Report",
        "eval_title": "Comparative Model Evaluation",
        "eval_description": "This section allows evaluating the models on the validation dataset and comparing metrics.",
        "generate_report_saved": "📄 Generate Comparative PDF Report from Saved Results",
        "download_comparative_pdf": "📄 Download Comparative PDF",
        "re_eval": "Do you want to re-evaluate the models?",
        "re_eval_button": "🔍 Evaluate Models and Generate Comparative Report",
        "eval_success": "Evaluation completed and report successfully generated!",
        "metrics_header": "📊 Metrics per Model",
        "comparison_header": "📈 Statistical Comparisons (McNemar)",
        "conf_matrix": "🧮 Confusion Matrices",
        "roc_pr": "📉 ROC and Precision-Recall Curves",
        "select_model": "Model to use",
        "confidence_threshold": "Confidence threshold for diagnosis",
        "processing_image": "Processing image...",
        "analyzing_image": "Analyzing image...",
        "generate_pdf_button": "📄 Generate and Download PDF Report",
        "generate_from_saved_button": "📥 Generate PDF from Saved Results",
        "evaluating_models": "Evaluating models...",
        "sidebar_title": "Navigation",
        "sidebar_go_to": "Go to",
        "model": "Model"
    },
        "Português": {
        "page_title": "Sistema de Diagnóstico de Câncer de Pulmão",
        "page_info": "🏥",
        "nav_diagnosis": "Diagnóstico por Imagem",
        "nav_eval": "Avaliação de Modelos",
        "upload_header": "Carregar Imagem",
        "upload_option": "Selecione o tipo de entrada",
        "upload_png": "Enviar imagem PNG",
        "upload_example": "Usar exemplo",
        "select_class": "Selecionar classe de exemplo",
        "select_image": "Selecionar imagem",
        "image_warning": "Nenhuma imagem encontrada na pasta",
        "model_warning": "Modelo não encontrado",
        "image_view": "Visualização da Imagem",
        "original_image": "Imagem Original Redimensionada",
        "preprocessed_image": "Imagem Pré-processada",
        "results": "Resultados do Diagnóstico",
        "prediction": "Predição",
        "confidence": "Confiança",
        "low_confidence": "Confiança baixa. Avaliação adicional recomendada.",
        "negative": "Nenhum sinal maligno detectado. Acompanhamento rotineiro recomendado.",
        "positive": "Possíveis sinais de câncer detectados. Consulte um especialista.",
        "saliency_map": "Mapa de Saliência (Regiões relevantes)",
        "generate_pdf": "Gerar Relatório em PDF",
        "download_pdf": "📄 Baixar Relatório PDF",
        "eval_title": "Avaliação Comparativa de Modelos",
        "eval_description": "Esta seção permite avaliar os modelos no conjunto de validação e comparar as métricas.",
        "generate_report_saved": "📄 Gerar Relatório Comparativo em PDF com Resultados Salvos",
        "download_comparative_pdf": "📄 Baixar PDF Comparativo",
        "re_eval": "Deseja reavaliar os modelos?",
        "re_eval_button": "🔍 Avaliar Modelos e Gerar Relatório Comparativo",
        "eval_success": "Avaliação concluída e relatório gerado com sucesso!",
        "metrics_header": "📊 Métricas por Modelo",
        "comparison_header": "📈 Comparações Estatísticas (McNemar)",
        "conf_matrix": "🧮 Matrizes de Confusão",
        "roc_pr": "📉 Curvas ROC e Precisão-Recall",
        "select_model": "Modelo a utilizar",
        "confidence_threshold": "Limite de confiança para diagnóstico",
        "processing_image": "Processando imagem...",
        "analyzing_image": "Analisando imagem...",
        "generate_pdf_button": "📄 Gerar e Baixar Relatório PDF",
        "generate_from_saved_button": "📥 Gerar PDF a partir de Resultados Salvos",
        "evaluating_models": "Avaliando modelos...",
        "sidebar_title": "Navegação",
        "sidebar_go_to": "Ir para",
        "model": "Modelo"
    },
    "Français": {
        "page_title": "Système de Diagnostic du Cancer du Poumon",
        "page_info": "🏥",
        "nav_diagnosis": "Diagnostic par Image",
        "nav_eval": "Évaluation des Modèles",
        "upload_header": "Téléversement d'Image",
        "upload_option": "Sélectionnez le type d'entrée",
        "upload_png": "Téléverser une image PNG",
        "upload_example": "Utiliser un exemple",
        "select_class": "Sélectionner une classe d'exemple",
        "select_image": "Sélectionner une image",
        "image_warning": "Aucune image trouvée dans le dossier",
        "model_warning": "Modèle non trouvé",
        "image_view": "Visualisation de l'Image",
        "original_image": "Image Originale Redimensionnée",
        "preprocessed_image": "Image Prétraitée",
        "results": "Résultats du Diagnostic",
        "prediction": "Prédiction",
        "confidence": "Confiance",
        "low_confidence": "Confiance faible. Évaluation supplémentaire recommandée.",
        "negative": "Aucun signe malin détecté. Suivi de routine recommandé.",
        "positive": "Signes possibles de cancer détectés. Consultez un spécialiste.",
        "saliency_map": "Carte de Saillance (Régions pertinentes)",
        "generate_pdf": "Générer un Rapport PDF",
        "download_pdf": "📄 Télécharger le Rapport PDF",
        "eval_title": "Évaluation Comparative des Modèles",
        "eval_description": "Cette section permet d'évaluer les modèles sur le jeu de validation et de comparer les métriques.",
        "generate_report_saved": "📄 Générer un Rapport Comparatif PDF à partir des Résultats Enregistrés",
        "download_comparative_pdf": "📄 Télécharger le PDF Comparatif",
        "re_eval": "Souhaitez-vous réévaluer les modèles ?",
        "re_eval_button": "🔍 Évaluer les Modèles et Générer le Rapport Comparatif",
        "eval_success": "Évaluation terminée et rapport généré avec succès !",
        "metrics_header": "📊 Métriques par Modèle",
        "comparison_header": "📈 Comparaisons Statistiques (McNemar)",
        "conf_matrix": "🧮 Matrices de Confusion",
        "roc_pr": "📉 Courbes ROC et Précision-Rappel",
        "select_model": "Modèle à utiliser",
        "confidence_threshold": "Seuil de confiance pour le diagnostic",
        "processing_image": "Traitement de l'image...",
        "analyzing_image": "Analyse de l'image...",
        "generate_pdf_button": "📄 Générer et Télécharger le Rapport PDF",
        "generate_from_saved_button": "📥 Générer un PDF à partir des Résultats Enregistrés",
        "evaluating_models": "Évaluation des modèles...",
        "sidebar_title": "Navigation",
        "sidebar_go_to": "Aller à",
        "model": "Modéle"
    }
}

pdf_text = {
    "Español": {
        "report_title": "Reporte de Diagnóstico Pulmonar",
        "model_used": "Modelo utilizado",
        "date": "Fecha",
        "diagnosis": "Diagnóstico",
        "positive": "Positivo para Cáncer",
        "negative": "Negativo para Cáncer",
        "confidence": "Confianza",
        "image_section": "Imágenes de Diagnóstico",
        "original": "Original",
        "saliency_map": "Mapa de Saliencia",
        "download_button": "📥 Descargar Reporte de Diagnóstico",
        "comparative_title": "Reporte Comparativo de Modelos",
        "dataset_info": "Este informe presenta una evaluación comparativa de modelos entrenados sobre el conjunto de datos: Lung Cancer MRI Dataset (Kaggle). Se utilizó un 80% de los datos para entrenamiento y un 20% para pruebas.",
        "execution_env": "Entorno de Ejecución",
        "hardware": "Hardware",
        "software": "Software",
        "evaluated_models": "Modelos Evaluados",
        "training_time": "Tiempo de entrenamiento",
        "best_model": "Mejor modelo",
        "general_metrics": "Métricas Generales",
        "headers": ["Modelo", "Precisión", "Sensibilidad", "Especificidad", "F1-Score", "MCC"],
        "stat_comparison": "Comparaciones Estadísticas (McNemar)",
        "comparison_label": "Comparación",
        "statistic": "Estadístico de McNemar",
        "conf_matrices": "Matrices de Confusión",
        "matrix_label": "Matriz"
    },
    "English": {
        "report_title": "Lung Diagnosis Report",
        "model_used": "Model used",
        "date": "Date",
        "diagnosis": "Diagnosis",
        "positive": "Positive for Cancer",
        "negative": "Negative for Cancer",
        "confidence": "Confidence",
        "image_section": "Diagnosis Images",
        "original": "Original",
        "saliency_map": "Saliency Map",
        "download_button": "📥 Download Diagnosis Report",
        "comparative_title": "Comparative Model Report",
        "dataset_info": "This report presents a comparative evaluation of models trained on the Lung Cancer MRI Dataset (Kaggle). 80% of the data was used for training and 20% for testing.",
        "execution_env": "Execution Environment",
        "hardware": "Hardware",
        "software": "Software",
        "evaluated_models": "Evaluated Models",
        "training_time": "Training Time",
        "best_model": "Best Model",
        "general_metrics": "General Metrics",
        "headers": ["Model", "Accuracy", "Sensitivity", "Specificity", "F1-Score", "MCC"],
        "stat_comparison": "Statistical Comparisons (McNemar)",
        "comparison_label": "Comparison",
        "statistic": "McNemar Statistic",
        "conf_matrices": "Confusion Matrices",
        "matrix_label": "Matrix"
    },
    "Português": {
        "report_title": "Relatório de Diagnóstico Pulmonar",
        "model_used": "Modelo utilizado",
        "date": "Data",
        "diagnosis": "Diagnóstico",
        "positive": "Positivo para Câncer",
        "negative": "Negativo para Câncer",
        "confidence": "Confiança",
        "image_section": "Imagens de Diagnóstico",
        "original": "Original",
        "saliency_map": "Mapa de Saliência",
        "download_button": "📥 Baixar Relatório de Diagnóstico",
        "comparative_title": "Relatório Comparativo de Modelos",
        "dataset_info": "Este relatório apresenta uma avaliação comparativa de modelos treinados com o conjunto de dados: Lung Cancer MRI Dataset (Kaggle). 80% dos dados foram usados para treinamento e 20% para testes.",
        "execution_env": "Ambiente de Execução",
        "hardware": "Hardware",
        "software": "Software",
        "evaluated_models": "Modelos Avaliados",
        "training_time": "Tempo de Treinamento",
        "best_model": "Melhor Modelo",
        "general_metrics": "Métricas Gerais",
        "headers": ["Modelo", "Acurácia", "Sensibilidade", "Especificidade", "F1-Score", "MCC"],
        "stat_comparison": "Comparações Estatísticas (McNemar)",
        "comparison_label": "Comparação",
        "statistic": "Estatística de McNemar",
        "conf_matrices": "Matrizes de Confusão",
        "matrix_label": "Matriz"
    },
    "Français": {
        "report_title": "Rapport de Diagnostic Pulmonaire",
        "model_used": "Modèle utilisé",
        "date": "Date",
        "diagnosis": "Diagnostic",
        "positive": "Positif au Cancer",
        "negative": "Négatif au Cancer",
        "confidence": "Confiance",
        "image_section": "Images de Diagnostic",
        "original": "Original",
        "saliency_map": "Carte de Saillance",
        "download_button": "📥 Télécharger le Rapport de Diagnostic",
        "comparative_title": "Rapport Comparatif des Modèles",
        "dataset_info": "Ce rapport présente une évaluation comparative des modèles entraînés sur le jeu de données : Lung Cancer MRI Dataset (Kaggle). 80% des données ont été utilisées pour l'entraînement et 20% pour les tests.",
        "execution_env": "Environnement d'Exécution",
        "hardware": "Matériel",
        "software": "Logiciel",
        "evaluated_models": "Modèles Évalués",
        "training_time": "Temps d'entraînement",
        "best_model": "Meilleur Modèle",
        "general_metrics": "Métriques Générales",
        "headers": ["Modèle", "Précision", "Sensibilité", "Spécificité", "F1-Score", "MCC"],
        "stat_comparison": "Comparaisons Statistiques (McNemar)",
        "comparison_label": "Comparaison",
        "statistic": "Statistique de McNemar",
        "conf_matrices": "Matrices de Confusion",
        "matrix_label": "Matrice"
    }
}

# **Métricas (metrics_utils.py)**

In [ ]:
%%writefile metrics_utils.py
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score
from scipy.stats import chi2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
import json

RESULTADOS_DIR = "/content/resultados"
os.makedirs(RESULTADOS_DIR, exist_ok=True)
JSON_DIR = os.path.join(RESULTADOS_DIR, "json")
os.makedirs(JSON_DIR, exist_ok=True)

def matthews_corrcoef(cm):
    tp, fp, fn, tn = cm[1][1], cm[0][1], cm[1][0], cm[0][0]
    numerator = (tp * tn) - (fp * fn)
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return numerator / denominator if denominator != 0 else 0

def mcnemar_test(y_true, y_model1, y_model2):
    table = np.zeros((2, 2))
    for true, pred1, pred2 in zip(y_true, y_model1, y_model2):
        if pred1 == true and pred2 != true:
            table[0][1] += 1
        elif pred1 != true and pred2 == true:
            table[1][0] += 1
    if table[0][1] + table[1][0] > 25:
        statistic = (np.abs(table[0][1] - table[1][0]) - 1) ** 2 / (table[0][1] + table[1][0])
    else:
        statistic = (np.abs(table[0][1] - table[1][0])) ** 2 / (table[0][1] + table[1][0])
    p_value = 1 - chi2.cdf(statistic, df=1)
    return statistic, p_value

def save_curves(model_name, y_true, y_scores, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model_safe = model_name.lower().replace(" ", "_")

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(f'Curva ROC - {model_name}')
    plt.legend()
    plt.tight_layout()
    roc_path = os.path.join(output_dir, f'roc_{model_safe}.png')
    plt.savefig(roc_path)
    plt.close()

    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    ap_score = average_precision_score(y_true, y_scores)
    plt.figure(figsize=(6,5))
    plt.plot(recall, precision, label=f'AP = {ap_score:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precisión')
    plt.title(f'Curva Precision-Recall - {model_name}')
    plt.legend()
    plt.tight_layout()
    pr_path = os.path.join(output_dir, f'pr_{model_safe}.png')
    plt.savefig(pr_path)
    plt.close()

    return roc_path, pr_path

def evaluate_on_dataset(test_dir):
    def load_data():
        cases, labels = [], []
        for label_name, label in [('no_cancer', 0), ('cancer', 1)]:
            folder = os.path.join(test_dir, label_name)
            for img_file in os.listdir(folder):
                img = np.load(os.path.join(folder, img_file))
                if len(img.shape) == 2:
                    img = np.expand_dims(img, axis=-1)
                cases.append(img)
                labels.append(label)
        return np.array(cases), np.array(labels)

    X, y_true = load_data()

    models = {
        "CNN 2D Personalizada": load_model("/content/models/cnn_2d_personalizada_lung_cancer.keras"),
        "DenseNet121": load_model("/content/models/densenet121_lung_cancer.keras"),
        "ResNet50": load_model("/content/models/resnet50_lung_cancer.keras")
    }

    metrics_list = []
    predictions_per_model = []
    confusion_matrices = []
    roc_paths = []
    pr_paths = []

    for name, model in models.items():
        if model.input_shape[-1] == 3:
            X_processed = np.repeat(X, 3, axis=-1)
        else:
            X_processed = X

        preds = model.predict(X_processed, batch_size=32).flatten()
        y_pred = (preds > 0.5).astype(int)
        predictions_per_model.append(y_pred)

        cm = confusion_matrix(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=['no_cancer', 'cancer'], output_dict=True)

        metrics_list.append({
            'model': name,
            'accuracy': report['accuracy'],
            'sensitivity': report['cancer']['recall'],
            'specificity': report['no_cancer']['recall'],
            'f1': report['cancer']['f1-score'],
            'mcc': matthews_corrcoef(cm)
        })

        plt.figure(figsize=(4, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                    xticklabels=['No Cáncer', 'Cáncer'],
                    yticklabels=['No Cáncer', 'Cáncer'])
        plt.title(f'Matriz de Confusión - {name}')
        plt.xlabel('Predicho')
        plt.ylabel('Real')

        cm_path = os.path.join(RESULTADOS_DIR, f'cm_{name.replace(" ", "_").lower()}.png')
        plt.tight_layout()
        plt.savefig(cm_path)
        plt.close()

        confusion_matrices.append(cm_path)

        roc_path, pr_path = save_curves(name, y_true, preds, RESULTADOS_DIR)
        roc_paths.append(roc_path)
        pr_paths.append(pr_path)

    comparisons = {}
    comparisons['0_1'] = mcnemar_test(y_true, predictions_per_model[0], predictions_per_model[1])
    comparisons['0_2'] = mcnemar_test(y_true, predictions_per_model[0], predictions_per_model[2])
    comparisons['1_2'] = mcnemar_test(y_true, predictions_per_model[1], predictions_per_model[2])

    metrics_json_path = os.path.join(JSON_DIR, "metrics.json")
    with open(metrics_json_path, "w") as f:
        json.dump(metrics_list, f, indent=4)

    comparisons_json_path = os.path.join(JSON_DIR, "comparisons.json")
    comparisons_str = {k: [float(v[0]), float(v[1])] for k, v in comparisons.items()}
    with open(comparisons_json_path, "w") as f:
        json.dump(comparisons_str, f, indent=4)

    paths_json_path = os.path.join(JSON_DIR, "image_paths.json")
    paths_data = {
        "confusions": confusion_matrices,
        "rocs": roc_paths,
        "prs": pr_paths
    }
    with open(paths_json_path, "w") as f:
        json.dump(paths_data, f, indent=4)

    return metrics_list, comparisons, confusion_matrices, roc_paths, pr_paths

# **Generar PDF (report_utils.py)**

In [ ]:
%%writefile report_utils.py
from fpdf import FPDF
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from translations import pdf_text
import time

RESULTADOS_DIR = "/content/resultados"
os.makedirs(RESULTADOS_DIR, exist_ok=True)

def generate_pdf_report(image_original, heatmap, diagnosis, confidence, model_name, lang="Español"):
    t = pdf_text.get(lang, pdf_text["Español"])

    combined_path = os.path.join(RESULTADOS_DIR, "combined_temp.png")
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    axes[0].imshow(image_original, cmap='gray')
    axes[0].set_title(t["original"])
    axes[0].axis('off')
    axes[1].imshow(image_original, cmap='gray')
    axes[1].imshow(heatmap, cmap='jet', alpha=0.5)
    axes[1].set_title(t["saliency_map"])
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(combined_path, dpi=300)
    plt.close()

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=t["report_title"], ln=1, align='C')
    pdf.ln(5)

    pdf.cell(200, 10, txt=f"{t['model_used']}: {model_name}", ln=1)
    pdf.cell(200, 10, txt=f"{t['date']}: {time.strftime('%Y-%m-%d %H:%M:%S')}", ln=1)
    pdf.cell(200, 10, txt=f"{t['diagnosis']}: {t['positive'] if diagnosis else t['negative']}", ln=1)
    pdf.cell(200, 10, txt=f"{t['confidence']}: {confidence*100:.2f}%", ln=1)
    pdf.ln(10)

    pdf.cell(200, 10, txt=t["image_section"], ln=1)
    pdf.image(combined_path, w=180)

    pdf_path = os.path.join(RESULTADOS_DIR, "diagnostico_pulmonar.pdf")
    pdf.output(pdf_path)

    with open(pdf_path, "rb") as f:
        st.download_button(t["download_button"], data=f, file_name=pdf_path, mime="application/pdf")

    os.remove(combined_path)
    os.remove(pdf_path)

def generate_comparison_report(metrics_list, model_names, comparisons, confusion_matrices, lang="Español", output_path=os.path.join(RESULTADOS_DIR, 'report_comparativo.pdf')):
    t = pdf_text.get(lang, pdf_text["Español"])

    pdf = FPDF()
    pdf.set_auto_page_break(auto=False)
    pdf.add_page()
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 12, t["comparative_title"], ln=1, align='C')
    pdf.ln(5)

    pdf.set_font("Arial", size=11)
    pdf.multi_cell(0, 9, t["dataset_info"])
    pdf.ln(6)

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, t["execution_env"], ln=1)
    pdf.ln(3)

    pdf.set_font("Arial", 'B', 11)
    pdf.cell(0, 9, t["hardware"] + ": ", ln=1)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 9, "- RAM: 12.7 GB", ln=1)
    pdf.cell(0, 9, "- CPU: Intel(R) Xeon(R) CPU @ 2.20GHz", ln=1)
    pdf.ln(4)

    pdf.set_font("Arial", 'B', 11)
    pdf.cell(0, 9, "Software:", ln=1)
    pdf.set_font("Arial", size=10)
    software_versions = [
        "Python: 3.11.13", "Streamlit: 1.46.1", "TensorFlow: 2.18.0", "Keras: 3.8.0",
        "Matplotlib: 3.10.0", "Seaborn: 0.13.2", "Pandas: 2.2.2", "NumPy: 2.0.2",
        "Scikit-learn: 1.6.1", "Pyngrok: 7.2.12", "FPDF2: 2.8.3"
    ]
    for sv in software_versions:
        pdf.cell(0, 9, f"- {sv}", ln=1)
    pdf.ln(5)

    training_times = ["1hr 40min", "2hr 10min", "2hr 50min"]
    best_model = "DenseNet121"
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, t["evaluated_models"], ln=1)
    pdf.ln(2)
    pdf.set_font("Arial", size=10)
    for i, name in enumerate(model_names):
        pdf.cell(0, 9, f"- {name} | {t['training_time']}: {training_times[i]}", ln=1)
    pdf.ln(2)

    pdf.set_font("Arial", 'B', 11)
    pdf.set_text_color(0, 102, 204)
    pdf.cell(0, 9, f"{t['best_model']}: {best_model}", ln=1)
    pdf.set_text_color(0, 0, 0)
    pdf.ln(5)

    pdf.add_page()

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, t["general_metrics"], ln=1)
    pdf.ln(2)
    pdf.set_font("Arial", 'B', 10)
    col_width = 32
    for h in t["headers"]:
        pdf.cell(col_width, 10, h, border=1, align='C')
    pdf.ln()
    pdf.set_font("Arial", size=10)
    for i, metrics in enumerate(metrics_list):
        pdf.cell(col_width, 10, model_names[i], border=1)
        pdf.cell(col_width, 10, f"{metrics['accuracy']:.3f}", border=1, align='C')
        pdf.cell(col_width, 10, f"{metrics['sensitivity']:.3f}", border=1, align='C')
        pdf.cell(col_width, 10, f"{metrics['specificity']:.3f}", border=1, align='C')
        pdf.cell(col_width, 10, f"{metrics['f1']:.3f}", border=1, align='C')
        pdf.cell(col_width, 10, f"{metrics['mcc']:.3f}", border=1, align='C')
        pdf.ln()
    pdf.ln(8)

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, t["stat_comparison"], ln=1)
    pdf.ln(2)
    pdf.set_font("Arial", size=10)

    for key, (stat, pval) in comparisons.items():
        m1, m2 = key.split('_')
        text = f"{model_names[int(m1)]} vs {model_names[int(m2)]}: {t['statistic']} = {stat:.4f}, p-value = {pval:.4f}"
        pdf.multi_cell(0, 8, text)
        pdf.ln(1)

    pdf.ln(4)

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, t["conf_matrices"], ln=1)
    pdf.ln(3)
    x_positions = [10, 110]
    y_position = pdf.get_y()

    for idx, cm_path in enumerate(confusion_matrices):
        col = idx % 2
        if idx != 0 and col == 0:
            y_position += 90
        pdf.set_xy(x_positions[col], y_position)
        pdf.image(cm_path, x=x_positions[col], y=y_position, w=80)

    pdf.output(output_path)
    return output_path

# **Aplicación Principal (app.py)**


In [ ]:
%%writefile app.py
import streamlit as st
import os
import matplotlib.pyplot as plt
from model_utils import predict_image, predict_image_rgb
from preprocessing import load_and_preprocess_image
from tensorflow.keras.models import load_model
from report_utils import generate_pdf_report, generate_comparison_report
from translations import app_text, pdf_text
from metrics_utils import evaluate_on_dataset
import time
import glob
import json

lang = st.sidebar.selectbox("🌐 Seleccionar idioma / Select Language", ["Español", "English", "Português", "Français"])
t = app_text.get(lang, app_text["Español"])
t2 = pdf_text.get(lang, pdf_text["Español"])

st.set_page_config(
    page_title=t["page_title"],
    page_icon=t["page_info"],
    layout="wide"
)

MODEL_PATHS = {
    "CNN 2D Personalizada": "/content/models/cnn_2d_personalizada_lung_cancer.keras",
    "DenseNet121": "/content/models/densenet121_lung_cancer.keras",
    "ResNet50": "/content/models/resnet50_lung_cancer.keras"
}
DATASET_PATH = "/content/drive/MyDrive/lung_cancer_MRI_dataset/validate"
JSON_DIR = "/content/resultados/json"

@st.cache_resource
def load_selected_model(model_name):
    model_path = MODEL_PATHS.get(model_name)
    if model_path and os.path.exists(model_path):
        return load_model(model_path)
    else:
        st.error(f"{t['model_warning']}: {model_path}")
        st.stop()

def load_saved_results():
    try:
        metrics_path = os.path.join(JSON_DIR, "metrics.json")
        comparisons_path = os.path.join(JSON_DIR, "comparisons.json")
        image_paths_path = os.path.join(JSON_DIR, "image_paths.json")

        if not all(os.path.exists(p) for p in [metrics_path, comparisons_path, image_paths_path]):
            return None

        with open(metrics_path, "r") as f:
            metrics_list = json.load(f)

        with open(comparisons_path, "r") as f:
            comparisons_raw = json.load(f)
        comparisons = {k: tuple(v) for k, v in comparisons_raw.items()}

        with open(image_paths_path, "r") as f:
            paths = json.load(f)

        return metrics_list, comparisons, paths["confusions"], paths["rocs"], paths["prs"]

    except:
        return None

def mostrar_resultados(metrics_list, comparisons, confusion_matrices, roc_paths, pr_paths):
    st.subheader(t["metrics_header"])
    for metrics in metrics_list:
        st.markdown(f"### {metrics['model']}")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Accuracy", f"{metrics['accuracy']:.3f}")
            st.metric("F1", f"{metrics['f1']:.3f}")
        with col2:
            st.metric(t["confidence"], f"{metrics['sensitivity']:.3f}")
            st.metric("Specificity", f"{metrics['specificity']:.3f}")
        with col3:
            st.metric("MCC (Matthews Correlation Coefficient)", f"{metrics['mcc']:.3f}")

    st.subheader(t["comparison_header"])
    for key, (stat, p_value) in comparisons.items():
        m1, m2 = key.split('_')
        name1 = list(MODEL_PATHS.keys())[int(m1)]
        name2 = list(MODEL_PATHS.keys())[int(m2)]
        st.write(f"**{name1} vs {name2}** — Statistical: `{stat:.4f}` | p-value: `{p_value:.4f}`")

    st.subheader(t["conf_matrix"])
    cols = st.columns(2)
    for i, cm_path in enumerate(confusion_matrices):
        with cols[i % 2]:
            st.image(cm_path, caption=f"{t['conf_matrix']} - {list(MODEL_PATHS.keys())[i]}", use_container_width=True)

    st.subheader(t["roc_pr"])
    for i, model_name in enumerate(MODEL_PATHS.keys()):
        st.markdown(f"### {model_name}")
        col1, col2 = st.columns(2)
        with col1:
            st.image(roc_paths[i], caption="ROC", use_container_width=True)
        with col2:
            st.image(pr_paths[i], caption="Precision-Recall", use_container_width=True)

st.sidebar.title(t["sidebar_title"])
page = st.sidebar.radio(t["sidebar_go_to"], [t["nav_diagnosis"], t["nav_eval"]])

if page == t["nav_diagnosis"]:
    st.title(t["page_title"])

    model_names = list(MODEL_PATHS.keys())
    selected_model_name = st.sidebar.selectbox(t["select_model"], model_names)

    confidence_threshold = st.sidebar.slider(
        t["confidence_threshold"],
        min_value=0.1, max_value=0.99, value=0.5, step=0.01
    )

    model = load_selected_model(selected_model_name)

    st.header(t["upload_header"])
    upload_option = st.radio(t["upload_option"], [t["upload_png"], t["upload_example"]])

    uploaded_file = None

    if upload_option == t["upload_png"]:
        uploaded_file = st.file_uploader(t["upload_png"], type=["png"])
    else:
        class_folders = [f for f in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, f))]
        selected_class = st.selectbox(t["select_class"], class_folders)
        selected_folder = os.path.join(DATASET_PATH, selected_class)
        class_images = glob.glob(os.path.join(selected_folder, "*.png"))

        if class_images:
            file_names = [os.path.basename(f) for f in class_images]
            file_choice = st.selectbox(t["select_image"], file_names)
            if file_choice:
                uploaded_file = os.path.join(selected_folder, file_choice)
            else:
                st.warning(t["image_warning"])
        else:
            st.warning(t["image_warning"])

    if uploaded_file:
        with st.spinner(t["processing_image"]):
            image_preprocessed, image_original = load_and_preprocess_image(uploaded_file)
            time.sleep(1)

        st.subheader(t["image_view"])
        col1, col2 = st.columns(2)
        with col1:
            st.image(image_original, caption=t["original_image"], use_container_width=True)
        with col2:
            st.image(image_preprocessed, caption=t["preprocessed_image"], use_container_width=True)

        st.header(t["results"])
        with st.spinner(t["analyzing_image"]):
            if selected_model_name in ["ResNet50", "DenseNet121"]:
                prediction, confidence, heatmap = predict_image_rgb(model, image_preprocessed)
            else:
                prediction, confidence, heatmap = predict_image(model, image_preprocessed)

        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric(t["model"], selected_model_name)
        with col2:
            st.metric(t["prediction"], t2["positive"] if prediction == 1 else t2["negative"])
        with col3:
            st.metric(t["confidence"], f"{confidence:.2%}")

        if confidence < confidence_threshold:
            st.warning(t["low_confidence"])
        else:
            if prediction == 0:
                st.success(t["negative"])
            else:
                st.error(t["positive"])

        st.subheader(t["saliency_map"])
        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(image_original, cmap='gray')
        ax.imshow(heatmap, cmap='jet', alpha=0.5)
        ax.axis('off')
        st.pyplot(fig)

        st.subheader(t["generate_pdf"])
        if st.button(t["download_pdf"]):
            generate_pdf_report(
                image_original=image_original,
                heatmap=heatmap,
                diagnosis=prediction,
                confidence=confidence,
                model_name=selected_model_name,
                lang=lang
            )

elif page == t["nav_eval"]:
    st.title(t["eval_title"])
    st.markdown(t["eval_description"])

    loaded_results = load_saved_results()

    if loaded_results:
        metrics_list, comparisons, confusion_matrices, roc_paths, pr_paths = loaded_results
        mostrar_resultados(*loaded_results)

        st.subheader(t["generate_report_saved"])
        if st.button(t["generate_from_saved_button"]):
            report_path = generate_comparison_report(
                metrics_list,
                list(MODEL_PATHS.keys()),
                comparisons,
                confusion_matrices,
                lang=lang
            )
            with open(report_path, "rb") as f:
                st.download_button(
                    label=t["download_comparative_pdf"],
                    data=f,
                    file_name="comparacion_modelos.pdf",
                    mime="application/pdf"
                )

    st.divider()
    st.markdown(f"### {t['re_eval']}")
    if st.button(t["re_eval_button"]):
        with st.spinner(t["evaluating_models"]):
            test_dir = "/content/processed_lung_dataset_npy/validate"
            metrics_list, comparisons, confusion_matrices, roc_paths, pr_paths = evaluate_on_dataset(test_dir)
            report_path = generate_comparison_report(
                metrics_list,
                list(MODEL_PATHS.keys()),
                comparisons,
                confusion_matrices,
                lang=lang
            )

        with open(report_path, "rb") as f:
            st.download_button(t["download_comparative_pdf"], data=f, file_name="comparacion_modelos.pdf", mime="application/pdf")
        st.success(t["eval_success"])

        mostrar_resultados(metrics_list, comparisons, confusion_matrices, roc_paths, pr_paths)

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = os.getenv("NGROK_TOKEN")

public_url = ngrok.connect(addr="8501", proto="http")
print(f"Tu app está en: {public_url}")

!streamlit run app.py